In [32]:
import os
os.getcwd()

'C:\\Users\\Chandan\\OneDrive - iitkgp.ac.in\\Desktop\\Python'

In [34]:
os.chdir('C:\\Users\\Chandan\\OneDrive - iitkgp.ac.in\\Desktop\\Python')
os.getcwd()

'C:\\Users\\Chandan\\OneDrive - iitkgp.ac.in\\Desktop\\Python'

In [36]:
import cv2
import pandas as pd
from ultralytics import YOLO

# Initializing YOLOv5su model
model = YOLO('yolov5su.pt')

# Path to video
video_path = 'dataset_video.mp4'
cap = cv2.VideoCapture(video_path)

# Output file for results
output_file = 'crowd_detection_log.csv'

# Frame buffer to track consecutive frames with crowds
frame_buffer = []
crowd_log = []

# Function to calculate distance between two points
def distance(point1, point2):
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

# Function to check for crowds
def check_crowd(persons, threshold=100):
    crowd_groups = []
    visited = set()  # A set to track which persons have already been added to a crowd

    for i, person1 in enumerate(persons):
        if i in visited:
            continue  # Skip persons already in a crowd group

        group = [person1]
        visited.add(i)

        for j, person2 in enumerate(persons):
            if i != j and j not in visited:  # Only check unvisited persons
                dist = distance(person1, person2)
                if dist < threshold:
                    group.append(person2)
                    visited.add(j)

        if len(group) >= 3:
            crowd_groups.append(group)

    return crowd_groups

# Processing the video frame by frame
frame_count = 0
print('Started')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Running detection on the frame
    results = model(frame, conf=0.15, classes=[0], verbose=False)  # Class 0 is for 'person'
    persons = [((box[0] + box[2]) // 2, (box[1] + box[3]) // 2) for box in results[0].boxes.xyxy.cpu().numpy()]
    
    print(f"Frame {frame_count}: {len(persons)} persons detected")
    
    # Check for crowds
    crowd_groups = check_crowd(persons, threshold=100)
    if crowd_groups:
        frame_buffer.append((frame_count, sum(len(group) for group in crowd_groups)))
    else:
        if len(frame_buffer) == 0 or frame_buffer[-1][1] > 0:
            frame_buffer.append((frame_count, 0))

    # If crowd persists for 10 consecutive frames
    if len(frame_buffer) >= 10 and all(c[1] > 0 for c in frame_buffer[-10:]):
        frame_num, person_count = frame_buffer[-10]
        crowd_log.append({'Frame': frame_num, 'PersonCount': person_count})
        print(f"Crowd detected at Frame {frame_num} with {person_count} persons.")
        frame_buffer.clear()  # Resetting buffer after logging the crowd

    frame_count += 1

cap.release()

# Saving the log to a CSV file
df = pd.DataFrame(crowd_log)
df.to_csv(output_file, index=False)

print(f"Crowd detection complete! Results saved to {output_file}.")


Started
Frame 0: 40 persons detected
Frame 1: 39 persons detected
Frame 2: 40 persons detected
Frame 3: 41 persons detected
Frame 4: 41 persons detected
Frame 5: 41 persons detected
Frame 6: 40 persons detected
Frame 7: 39 persons detected
Frame 8: 37 persons detected
Frame 9: 37 persons detected
Crowd detected at Frame 0 with 12 persons.
Frame 10: 37 persons detected
Frame 11: 38 persons detected
Frame 12: 39 persons detected
Frame 13: 37 persons detected
Frame 14: 36 persons detected
Frame 15: 38 persons detected
Frame 16: 38 persons detected
Frame 17: 35 persons detected
Frame 18: 35 persons detected
Frame 19: 34 persons detected
Crowd detected at Frame 10 with 3 persons.
Frame 20: 34 persons detected
Frame 21: 36 persons detected
Frame 22: 34 persons detected
Frame 23: 35 persons detected
Frame 24: 35 persons detected
Frame 25: 37 persons detected
Frame 26: 39 persons detected
Frame 27: 37 persons detected
Frame 28: 34 persons detected
Frame 29: 34 persons detected
Crowd detected a